In [1]:
pip install pyevmasm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
import requests
import certifi
import time
from pyevmasm import instruction_tables, disassemble_hex, disassemble_all, assemble_hex
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#addresses = pd.read_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/addresses.csv")
#left_ds = pd.read_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/Dataset/dataset.csv")


left_ds = pd.read_csv("/content/ds_merged.csv",delimiter= ";")
addresses=left_ds["address"] 

In [3]:
addr_list = []
bytecode_list = {}
opcode_list = {}

In [4]:
for addr in left_ds['address']: #creo una lista con tutti gli address
      addr_list.append(addr)

In [5]:
for addr in addr_list: # ciclo su tutti gli indirizzi presenti nella lista
    normal_tx_url = "https://api.etherscan.io/api?module=account&action=txlist&address=" + addr + \
                   "&startblock=0&endblock=99999999&page=1&offset=10000&sort=asc&apikey=SAKUKN5G8E7R2IYIWI2IVS7ZRKXDAX8NE6"
    response_normal = requests.get(normal_tx_url, verify = certifi.where()) # la get restituisce un Response obj
    address_content_normal = response_normal.json() # la risposta e' in formato json, si ottiene un dizionario python
    result_normal = address_content_normal.get('result') # facendo la get e si ottiene tutto cio' che sta dopo result, si ottiene una lista

    for t in result_normal:
      if (t['to'] == ''): # prendo l'oggetto json con la creazione del contratto che ha il to = ''
        bytecode = (t['input']) # aggiungo il valore della chiave input che è uguale al bytecode
        bytecode_list[addr]=bytecode # faccio append con l'obiettivo di creare una lista con tutti i bytecode.
        time.sleep(0.1)
       

In [7]:
# creo una lista di bytecode
for k in bytecode_list.keys():
    opcode = disassemble_hex(bytecode_list[k])# conversione bytecode in opcode
    opcode_list[k] = opcode

In [8]:
df_full_comm = pd.DataFrame()
for addr in opcode_list.keys():
    row = []
    list_comm = pd.Series(opcode_list[addr].split('\n')) 
    list_comm = list_comm.str.replace("PUSH.*", "PUSH")
    list_comm = list_comm.str.replace("DUP.*", "DUP")
    list_comm = list_comm.str.replace("SWAP.*", "SWAP")
    list_comm = list_comm.str.replace("LOG.*", "LOG")
    df_comm = pd.DataFrame(list_comm.value_counts())
    df_comm = df_comm.T
    df_comm['address'] = [addr]
    df_full_comm = pd.concat([df_full_comm,df_comm] )

In [ ]:
#df_full_comm.to_csv(r"/Users/lucapennella/Desktop/Smart Ponzi/bytecode_decode.csv" , index=False)

In [11]:
df_full

,address,target,PUSH,INVALID,DUP,JUMPDEST,STOP,MSTORE,JUMPI,REVERT,...,SGT,DIFFICULTY,SMOD,BYTE,SIGNEXTEND,CHAINID,SELFBALANCE,GASPRICE,EXTCODEHASH,SDIV
0,0x0006157838d5a6b33ab66588a6a693a57c869999,0.0,13.0,10.0,3.0,3.0,3.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x001a589dda0d6be37632925eaf1256986b2c6ad0,0.0,1040.0,4.0,1126.0,90.0,2.0,166.0,86.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x00674045bb7c17f0aa1cde34780d6c51af548728,0.0,1175.0,24.0,720.0,340.0,151.0,159.0,165.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x006bea43baa3f7a6f765f14f10a1a1b08334ef45,0.0,826.0,49.0,570.0,216.0,8.0,133.0,103.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x00a9f7d093c46d95f0318e4a6ffc6ed68f73044c,0.0,660.0,63.0,353.0,188.0,211.0,60.0,102.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,0xffc8ab40d1255ee3fa23fbf2ff1628c4967105f2,0.0,559.0,14.0,252.0,198.0,124.0,25.0,82.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7998,0xffd883e94f7a3700aaf81a411bd164ad27acc656,0.0,2128.0,91.0,1656.0,324.0,50.0,271.0,242.0,124.0,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7999,0xffe8196bc259e8dedc544d935786aa4709ec3e64,0.0,779.0,36.0,618.0,181.0,6.0,147.0,103.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8000,0xfff18893bc6c430741b3c5ad483391a5e21220bb,0.0,859.0,7.0,484.0,158.0,7.0,132.0,65.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [12]:
df_full = pd.merge(left_ds, df_full_comm, how='left', on = 'address')

In [18]:
df_full.to_csv("/Users/lucapennella/Desktop/Smart Ponzi/bytecode_decode.csv" , index=False)

OSError: ignored